<a href="https://colab.research.google.com/github/saurabhIU/Deep-Learning/blob/master/Assignment_1_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import warnings
import time
import os
warnings.filterwarnings('ignore')

In [11]:
!ls

checkpoint			   my_test_model.index	sample_data
my_test_model.data-00000-of-00001  my_test_model.meta


In [12]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/MNIST_data", one_hot=True)



Extracting /tmp/data/MNIST_data/train-images-idx3-ubyte.gz
Extracting /tmp/data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
X_train, Y_train = mnist.train.images,mnist.train.labels
X_test, Y_test = mnist.test.images, mnist.test.labels
print (f'Total Training Images in Dataset {X_train.shape} , Total training label is {Y_train.shape}')
print (f'Total Test Images in Dataset {X_test.shape} , Total test label is {Y_test.shape}')

Total Training Images in Dataset (55000, 784) , Total training label is (55000, 10)
Total Test Images in Dataset (10000, 784) , Total test label is (10000, 10)


In [0]:
EPOCHS = 400
BATCH_SIZE = tf.placeholder(tf.int64)
TEST_BATCH_SIZE = tf.placeholder(tf.int64)
training_dataset = tf.data.Dataset.from_tensor_slices((X_train, tf.cast(Y_train,tf.float32))).shuffle(500).repeat().batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, tf.cast(Y_test,tf.float32))).batch(TEST_BATCH_SIZE)

In [0]:
iterator = tf.data.Iterator.from_structure(training_dataset.output_types,
                                               training_dataset.output_shapes)
next_batch = iterator.get_next()

In [0]:
training_initialise = iterator.make_initializer(training_dataset)
test_initialise = iterator.make_initializer(test_dataset)

In [0]:
def nn_model(input_data):
 
    layer1 = tf.layers.dense(input_data, 1024,activation=tf.nn.relu)
    layer2 = tf.layers.dense(layer1, 1024,activation=tf.nn.relu)
    layer3 = tf.layers.dense(layer2, 1024,activation=tf.nn.relu)
    layer4 = tf.layers.dense(layer3, 1024,activation=tf.nn.relu)
    logits = tf.layers.dense(layer4, 10)
    
    return logits



In [0]:
logits = nn_model(next_batch[0])
cost = tf.nn.softmax_cross_entropy_with_logits_v2(labels=next_batch[1],logits=logits)
optimizer = tf.train.AdamOptimizer().minimize(cost)
prediction = tf.nn.softmax(logits)
correct_prediction = tf.equal(tf.argmax(prediction, axis=1), tf.argmax(next_batch[1], axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()
save_dir = 'checkpoints/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)


In [21]:
sess =  tf.Session() 
    
sess.run(tf.global_variables_initializer())
sess.run(training_initialise,feed_dict={BATCH_SIZE : 5000 })
  
tic = time.time()
for i in range(EPOCHS):
    c,_,acc = sess.run([cost, optimizer, accuracy])
    if i % 100 == 0:
      print(f'Epoch: {i},training accuracy:{acc * 100}')
toc = time.time()
print(f'Time taken for training is {toc-tic}')
saver.save(sess, save_path=save_dir)
#   print(sess.run('dense/kernel:0'))
sess.run(test_initialise,feed_dict={TEST_BATCH_SIZE : 10000 })
print(f'Test Accuracy is {sess.run(accuracy*100)}')

Epoch: 0,training accuracy:8.919999748468399
Epoch: 100,training accuracy:99.23999905586243
Epoch: 200,training accuracy:99.95999932289124
Epoch: 300,training accuracy:99.86000061035156
Time taken for training is 33.39911437034607
Test Accuracy is 97.50999450683594


**Once you’re done with training, as a starter, do a feedforward step on your test samples, a thousand of them. Capture the output of the softmax layer, which will be a 10-dim probability vector per sample. In other words, each output dimension has 1,000 predictions corresponding to the 1,000 examples. For each 10-d output vector, find the dim with the maximum probability (which will eventually decide the class label). Plot the input image associated with that in a grid of subplots. For example, you can create a 10 × 10 grid of subplots, whose first row plots first ten input images that produced the highest probabilities for the first dim (which corresponds to “0”). Eventually, if your classification was near perfect, you’ll see ten 0’s in the first row, ten 1’s in the second, and so on.**

In [25]:
#Lets take a sample of 1000 from test set
saver.restore(sess, save_path=save_dir)
sess.run(tf.global_variables_initializer())
sess.run(test_initialise,feed_dict={TEST_BATCH_SIZE : 1000 })
print(f'Test Accuracy is {sess.run(accuracy*100)}')
#   output = sess.run(prediction)
#   print(output.shape)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoints/


RuntimeError: ignored